In [1]:
### Import libraries
import openai
import random
import os
import time
from datetime import datetime
import pandas as pd
import clingo
from clingo.control import Control
import json
from helper import *
from stories import *
from pipelines import *
import psycopg2

In [2]:
now = datetime.now()
date_time = now.strftime("%Y%m%d")

## OpenAI

In [3]:
set_api_key()

## Connect to Database

In [4]:
conn = get_connection()

## Load Projects

In [5]:
projects = get_useful_projects()

## Functions

In [6]:
def save_in_database(conn, prompt1, prompt2, encoding, story, story_1, story_2, version, file):
    cursor = conn.cursor()
    command = "INSERT INTO ne_linebyline (prompt1, prompt2, encoding, story, story_1, story_2, version, file) VALUES ('%(prompt1)s', '%(prompt2)s', '%(encoding)s', '%(story)s', '%(story_1)s', '%(story_2)s', '%(version)s', '%(file)s');" % {"prompt1": prompt1, "prompt2": prompt2, "encoding": encoding, "story": story, "story_1": story_1, "story_2":story_2, "version":version, "file":file}
    try:
        cursor.execute(command)
        conn.commit()
        cursor.close()
    except Exception as err:
        print(command)
        print(f"Unexpected {err=}, {type(err)=}")
        conn.commit()
        cursor.close()
        pass

In [7]:
def get_2_random_projects(projects,project):
    filtered_projects = [element for element in projects if element != project]
    project_1, project_2 = random.sample(filtered_projects,2)
    return project_1, project_2

In [8]:
def quality_check(name_file, lines_file, previous_messages, instance, deep):
    if deep==0:
        return lines_file[-1]
    with open(name_file,"w") as tmp_file:
        for new_line in lines_file:
            tmp_file.write(new_line)
    models, errors, symbols, mistakes = asp_try_5(name_file,instance)
    #print(f"Errors {errors}. Messages {mistakes}.")
    new_answer = lines_file[-1]
    if len(mistakes) > 0:
        print("Mistake! Last Message : " + str(previous_messages[-1]))
        answer = get_completion_line('''The last rule that you wrote is wrong. The error message is: 
                                    ###" + str(mistakes) + "###.
                                    Write that last rule again, correcting it, without apologizing or writing any explanation.'''
                                    , previous_messages = previous_messages)
        lines_file[-1] = answer
        print("Deep: "+ str(deep) + ". New answer: " +answer)
        new_answer = quality_check(name_file,lines_file,previous_messages, instance, deep-1)
    return new_answer

## Main : From Script, Line4Line, ICL, TryAgain

In [11]:
for project in projects:
    story = project["story"]
    encoding = project["lines"]
    instance = instances_dict[story]
    problem = project["problem"]
    lines = encoding.splitlines()
    new_file = []
    project_1, project_2 = get_2_random_projects(projects,project)
    story_1 = project_1["story"]
    story_2 = project_2["story"]
    content1 = f'''Problem1: {project_1["problem"]}
                    Encoding1: {project_1["lines"]}  '''
    content2 = f'''Problem2: {project_2["problem"]}
                    Encoding2: {project_2["lines"]}  '''
    messages = [{"role":"user","content":content1}]
    messages.append({"role":"user","content":content2})
    messages.append({"role":"user","content":f"Problem1: {problem}"})
    new_file_name = story + "_from_" + story_1 + "_" + story_2 + "_v14_SLIT_" + date_time + ".lp"
    bad_file = False
    for line in lines:
        if line == '\n':
            continue
        if len(line.strip()) > 0 and line.strip()[0] == '%':
            print("---------------------------------------------------")
            new_file.append("\n" + line)
            messages.append({"role":"user","content":line})
            print(line)
            answer = get_completion_line(line,previous_messages=messages)
            print(answer)
            new_file.append("\n" + answer)
            messages.append({"role":"assistant","content":answer})
            if not bad_file:
                new_answer = quality_check(new_file_name,new_file,messages,instance,deep=3)
                new_file[-1] = new_answer
                with open(new_file_name,"w") as tmp_file:
                    for new_line in new_file:
                        tmp_file.write(new_line)
                models, errors, symbols, mistakes = asp_try_5(new_file_name,instance)
                if len(mistakes) > 0:
                    bad_file = True
    os.remove(new_file_name) 
    prompt = '''Given the description of a problem, and a commented line, 
                your task is to translate this commented line in Answer Set Programming. 
                Just write one or a few predicates or rules. Do not try to solve the whole problem at once.'''
    encoding = ' '.join(new_file)
    save_in_database(conn,prompt,'',encoding.replace("'",""),story,story_1, story_2, 14, new_file_name)  

---------------------------------------------------
% The possible next movements of the player "next" of 4 variables, are defined as 2 adjacent fields, with "field" predicate of 2 variables, that do not have a "wall" (4 variables) between them)          
next(X1,Y1,X2,Y2) :- field(X1,Y1), field(X2,Y2), adjacent(X1,Y1,X2,Y2), not wall(X1,Y1,X2,Y2).
---------------------------------------------------
            % A "field" is "next" to itself 
next(X,Y,X,Y) :- field(X,Y).
---------------------------------------------------
            % If a field is next to another, then this second one is next to the first one
next(X2,Y2,X1,Y1) :- next(X1,Y1,X2,Y2).
---------------------------------------------------
            % The player, "at" 3 variables, is at start position "start" 2 Variables, at timestep 0
at(X,Y,0) :- start(X,Y).
---------------------------------------------------
            % The player chooses only one position from the possible "next" fields if it is not at the "goal" 2

In [ ]:
new_file